# seedbleniding

exp40,exp49 のseed42,614


In [14]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

In [21]:
train_42 = pd.read_csv("../output/exp43_xgb_lagdiff_c3_hypar/oof_exp43_xgb_0.7956869117243499_5fold_seed42.csv")
test_42 = pd.read_csv("../output/exp43_xgb_lagdiff_c3_hypar/test_exp43_xgb_0.7956869117243499_5fold_seed42.csv")

train_611 = pd.read_csv("../output/exp43_xgb_611/oof_exp43_xgb_0.7940193621848979_5fold_seed611.csv")
test_611 = pd.read_csv("../output/exp43_xgb_611/test_exp43_xgb_0.7940193621848979_5fold_2_seed611.csv")



In [22]:
train_42.head()

,Unnamed: 0,customer_ID,target,tabnet_oot
0,0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.786225
1,1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.011697
2,2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.997550
3,3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.001872
4,4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001381


In [23]:
train_611.head()

,customer_ID,target,exp43_xgb_oof
0,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.001927
1,0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba17...,1,0.762834
2,0003e58375faf90552b2861c1ccea4ee5757827cbb6ecd...,0,0.009362
3,0004837f0c785928a29a6f83f70f4a1c54caec483a773f...,0,0.293286
4,00050d84c6d26e26cd2b18c3eed83d3130c270e2361470...,0,0.001449


In [24]:
oof = train_42.merge(train_611,on = "customer_ID")
sub = test_42.merge(test_611,on = "customer_ID")


In [25]:
sub.head()

,customer_ID,prediction,exp43_xgb_sub
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.027302,0.031097
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000748,0.000889
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.032421,0.029894
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.226533,0.227289
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.895852,0.881339


In [26]:
oof.head()

,Unnamed: 0,customer_ID,target_x,tabnet_oot,target_y,exp43_xgb_oof
0,0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.786225,1,0.811201
1,1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.011697,0,0.010708
2,2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.997550,1,0.996767
3,3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.001872,0,0.001518
4,4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001381,0,0.001314


In [29]:

oof["exp43_seedblend_oof"] = (oof["tabnet_oot"] + oof["exp43_xgb_oof"])/2
sub["exp43_seedblend_sub"] = (sub["prediction"] + sub["exp43_xgb_sub"])/2


In [30]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


score = amex_metric(oof["target_x"], oof["exp43_seedblend_oof"])
print("CV : ",score)

CV :  0.7949781307720436


In [31]:
oof[["customer_ID","exp43_seedblend_oof"]].to_csv(f"exp43_oof_xgb_{score}_seedblend.csv",index = False)
sub[["customer_ID","exp43_seedblend_sub"]].to_csv(f"exp43_sub_lxgb_{score}_seedblend.csv",index = False)


In [33]:

amex_metric(oof["target_x"], oof["tabnet_oot"])

0.795094661446722